In [ ]:
# Install the required packages for the project

%pip install -r requirements.txt

In [ ]:
# Ready the environment variables from the .env file

import os
from dotenv import load_dotenv

load_dotenv(override=True) 

TOKENS = os.getenv("TOKENS", "").split(",")

In [ ]:
# Define constants for API usage

API_URL = 'https://api.github.com'

LIMIT_REQUESTS_BY_TOKEN_PER_HOUR = 5000
LIMIT_REQUESTS_BY_IP_PER_HOUR = 60

In [ ]:
# Set API request utils (RUN IT ONLY ONCE TO KEEP COUNTER SYNCED)

import requests

# Initialize the counter for each token
counter = {token: LIMIT_REQUESTS_BY_TOKEN_PER_HOUR for token in TOKENS}
counter.update({None: LIMIT_REQUESTS_BY_IP_PER_HOUR})

# Get the next avaliable token to use for the request
def get_avaliable_token(): 
    token = None
    for t in counter:
        if counter[t] > 0:
            token = t
            break
        else:
            invalidate_token(t)
    
    return token
    
# Invalidate a token if it results in a error, avoiding to use it again
def invalidate_token(token):
    if token in counter:
        del counter[token]
        print(f"Token {token} has been invalidated.")
    else:
        print(f"Token {token} not found in the list.")

# Mark a token as used, reducing the counter
def used_token(token):
    if token in counter:
        counter[token] -= 1
        print(f"Token {token} used, remaining requests: {counter[token]}")
        if counter[token] == 0:
            invalidate_token(token)
    else:
        print(f"Token {token} not found in the list.")

# Function to make a request to the API
def request (path):
    # Set the URL to the API endpoint
    url = f'{API_URL}/{path}'
    if (path.startswith(API_URL)):
        url = path

    # Define headers with the token
    token = get_avaliable_token()
    headers = {}
    if token is not None:
        headers['Authorization'] = f'Bearer {token}'

    # Make the GET request
    response = requests.get(url, headers=headers)
    if response.ok:
        print(f"Request successful for token: {token}")
        used_token(token)
    else:
        if response.status_code == 401:
            print(f"Unauthorized access for token: {token}")
            used_token(token)

        elif response.status_code == 403:
            print(f"Rate limit exceeded for token: {token}")
            invalidate_token(token)
            # Retry with the next available token
            if (token is not None):
                print("Retrying with the next available token...")
                return request(path)
            
        elif response.status_code == 404:
            print("Resource not found")

        elif response.status_code == 429:
            print("Rate limit exceeded for IP")
            invalidate_token(token)

    return response

In [ ]:
# Single request example

response = request('/users')

if response.ok:
    print(response.json())
else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
# Multiple requests example

from time import sleep

# Testing the rate limit for a anonymous user per hour

for i in range(1, LIMIT_REQUESTS_BY_IP_PER_HOUR):
    response = request('/users')
    if response.ok:
        print(f"Request {i} succeeded!")
        print(response.json())
    else:
        print(f"Request {i} failed with status code {response.status_code}")
        break
    sleep(0.25)